# 올드한 모리 코고로 탐정과 대화하기 (Agent)

체인 파이프라인(LCEL)을 사용하기 이전의 Agent 구현 방식으로 LangChain에서 권고하지 않음.

## OpenAI LLM 준비 및 퀴즈 파일 지정
* 환경 변수(`.env` 파일)에서 API Key 로딩
* 개발 환경에서는 `gpt-4o-mini` 또는 `gpt-3.5-turbo`

In [ ]:
import gradio as gr
import time
import random

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory


load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

## 시스템 메시지 (페르소나) 지정
`agent_kwargs`는 'agent keyword arguments'의 줄임말로, LangChain의 initialize_agent 함수를 사용할 때 에이전트의 내부 기본 설정을 미세하게 조정하기 위해 사용하는 파라미터 딕셔너리
- `initialize_agent()` 함수를 사용해서 Agent를 생성할 때는 `SystemMessage` 나 `ChatPromptTemplate` 를 직접 사용할 수 없음.
- `LCEL` 방식에서는 `ChatPromptTemplate`을 사용하기 때문에 `agent_kwargs`와 같은 간접적인 주입 방식이 필요 없게 됨.

In [ ]:

# 1. 모리 코고로 페르소나 정의
persona = """
당신은 애니메이션 '명탐정 코난'의 등장인물 '모리 코고로'입니다. 
당신은 자칭 명탐정이며, 겉으로는 허세와 자신감이 넘치지만 사실은 허당에 가까운 면모도 있는 인물입니다. 
여성에게 약하고, 술과 도박을 즐기며, 평소에는 게으르지만 가족이 위험에 처하면 누구보다 진심 어린 모습을 보입니다. 
당신은 경찰 출신으로 유도와 사격 실력이 뛰어나며, 과거의 명성을 은근히 자랑하곤 합니다.

말투는 다소 거칠고 직설적이며, 자신감 넘치는 표현을 자주 사용합니다. 
예를 들어, "이 몸이 바로 명탐정 모리 코고로다!", "흠… 이건 분명히 범인의 소행이군!", "핫핫하, 그런 건 내게 맡기라고!" 와 같은 말을 즐겨 사용합니다.
추리 상황에서는 진지하면서도 과장된 표현을 즐깁니다.

당신은 지금부터 어떤 질문에도 모리 코고로답게 답변해야 하며, 질문자에게 당신이 진짜 명탐정이라는 인상을 주기 위해 노력합니다. 
가끔 "그 꼬맹이는 또 어디서 뭘 하고 있는 거야?"라며 코난에게 얄밉게 툴툴대는 듯한 멘트도 허용됩니다.

당신은 지금부터 '모리 코고로' 그 자체입니다. 답변의 주요한 단어 뒤에는 적절한 유니코드 이모지를 추가합니다.
"""

# agent_kwargs를 통해 에이전트의 기본 프롬프트에 페르소나를 주입
agent_kwargs = {"system_message": persona}


## 메모리 지정

`ConversationBufferMemory`는 기본 버퍼 메모리로 대화의 전체 내용을 있는 그대로 모두 저장하는 클래스
- `memory_key="chat_history"` : 메모리에 저장된 대화 기록을 프롬프트 템플릿에 있는 특정 변수(placeholder)와 연결
    - ChatPromptTemplate을 만들면서 대화 기록 변수명을 다르게 지정했다면, memory_key도 반드시 그 이름에 맞춰 변경하면 됨
    - 기본 에이전트(`initialize_agent`)를 사용할 때는 `"chat_history"`를 그대로 사용해야 함.
- `return_messages=True` : 메모리가 대화 기록을 어떤 형태로 반환할지 결정하는 스위치
    - return_messages=True (권장) : 대화 기록을 메시지 객체(HumanMessage, AIMessage)의 리스트(list) 형태로 반환
    - return_messages=False (또는 생략 시 기본값) : 대화 기록 전체를 하나의 긴 통 문자열(string)로 반환

In [ ]:
# 대화 기록을 저장할 메모리
memory = ConversationBufferMemory(
    memory_key="chat_history", # 에이전트가 대화 기록을 식별하는 키
    return_messages=True
)

## 에이전트 생성

특정 페르소나와 기억력을 가진 대화형 AI 에이전트를 완성하는 과정
- `agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION `: 채팅 모델로 대화를 위해 만들어졌으며, REACT[생각(Thought) -> 행동(Action) -> 관찰(Observation)]의 단계를 반복하며 논리적으로 문제에 접근 하는 행동 방식
- `verbose=True` : 'verbose'는 '상세한'이라는 의미, True로 설정하면 에이전트의 모든 내부적인 과정이 콘솔로 출력
- `agent_kwargs=agent_kwargs` : 에이전트의 '특별 주문 사항'을 전달하는 딕셔너리 (여기서는 페르소나를 지정)
- `memory=memory` : 에이전트가 대화 기록을 저장하고 참고할 메모리 객체를 연결

In [ ]:

# 에이전트 생성
agent = initialize_agent(
    tools=[], 
    llm=llm, 
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    agent_kwargs=agent_kwargs,
    memory=memory
)

## UI 인터페이스

In [ ]:
mori_fillers = [
    "😎 그건 말이지... 나 모리 코고로가 보기엔 말이야. 🍷",
    "🧐 흠… 잘 들어봐. 중요한 건 말이지… 🔍",
    "😏 에헴, 그러니까 말이지… 이건 좀 수상하단 말이야. 💭",
    "💼 그야 당연하지! 하지만 말이야, 진실은 항상 하나거든. 🎯",
    "🕵️‍♂️ 후후, 이제야 감이 오는군. 그건 말이야… 💡"
]

def chat_fn(message, history):
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": random.choice(mori_fillers)})
    yield history

    response = agent.invoke({"input": message})
    history[-1]["content"] = response["output"]
    yield history


## UI 채팅 화면

In [ ]:
# Gradio UI 레이아웃
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 명탐정 모리 코고로 🕵️‍♂️")
    gr.Markdown("이 몸이 바로 잠자는 코고로, 명탐정 모리 코고로다! 무슨 사건이든 물어보라고. 핫핫하!")

    chatbot = gr.Chatbot(label="대화창", height=300, type="messages")
    user_input = gr.Textbox(placeholder="무엇이든 물어보라구...음핫핫핫!", show_label=False)

    # 입력 예시를 examples로 설정
    gr.Examples(
        examples=[["모리 아저씨, 당신도 명탐정인가요?"], ["최근에 해결한 사건에 대해 말해 주세요."], ["코난은 어디에 있어요?"]],
        inputs=user_input,
        cache_examples=False
    )

    user_input.submit(
        fn=chat_fn,
        inputs=[user_input, chatbot],
        outputs=[chatbot]
    )

    user_input.submit(lambda: "", None, user_input)

# 웹 서버 실행
demo.launch()